In [3]:
#!pip3 install numba

In [4]:
# NOTE: The order of train set and test set should be reversed 

In [5]:
import os
import tensorflow as tf

# suppress info and warnings outputted by tensorflow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
# enable memory growth for gpu devices
# source: https://stackoverflow.com/a/55541385/8849692
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if gpu_devices:
    for device in gpu_devices:
        tf.config.experimental.set_memory_growth(device, True)

In [6]:
!nvidia-smi

Sat Feb 11 18:28:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 522.06       Driver Version: 522.06       CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   39C    P8     2W /  N/A |     44MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
!pip install "tensorflow-text>=2.10"
!pip install einops

In [8]:
import numpy as np

import typing
from typing import Any, Tuple

import einops
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import tensorflow as tf
import tensorflow_text as tf_text
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [9]:
#@title
class ShapeChecker():
  def __init__(self):
    # Keep a cache of every axis-name seen
    self.shapes = {}

  def __call__(self, tensor, names, broadcast=False):
    if not tf.executing_eagerly():
      return

    parsed = einops.parse_shape(tensor, names)

    for name, new_dim in parsed.items():
      old_dim = self.shapes.get(name, None)
      
      if (broadcast and new_dim == 1):
        continue

      if old_dim is None:
        # If the axis name is new, add its length to the cache.
        self.shapes[name] = new_dim
        continue

      if new_dim != old_dim:
        raise ValueError(f"Shape mismatch for dimension: '{name}'\n"
                         f"    found: {new_dim}\n"
                         f"    expected: {old_dim}\n")

In [10]:
def load_data(path1, path2):
  # text1 = path1.read_text(encoding='utf-8')
  # text2 = path2.read_text(encoding='utf-8')
  # with open(path1, "r", encoding="utf-8") as f:
  #   lines1 = f.readlines()
  #   # f=open("filename.txt","r",encoding='utf-8')
  #   # lines1 = text1.splitlines()
  # with open(path2) as f:
  #   lines2 = f.readlines()
  #   # lines2 = text2.splitlines(encoding='utf-8')
  # # pairs = [line.split('\t') for line in lines]

  # target = np.array([x.strip("\n") for x in lines1])
  # context = np.array([x.strip("\n") for x in lines2])

  

  return np.array(open(path1, encoding='utf-8').\
         read().strip().split('\n')),np.array(open(path2, encoding='utf-8').\
         read().strip().split('\n'))

In [11]:
target_raw, context_raw = load_data("informal", "formal")

In [12]:
print(context_raw[-1])
print(target_raw[-1])

I do have it. Please send me a message and I will send it to you.
I have it if you would like it... message me and I'll gladly send it to you!!


In [13]:
BUFFER_SIZE = len(context_raw)
BATCH_SIZE = 16

is_train = np.random.uniform(size=(len(target_raw),)) < 0.8

train_raw = (
    tf.data.Dataset
    .from_tensor_slices((context_raw[is_train], target_raw[is_train]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))
val_raw = (
    tf.data.Dataset
    .from_tensor_slices((context_raw[~is_train], target_raw[~is_train]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))

In [14]:
for example_context_strings, example_target_strings in train_raw.take(1):
  print(example_context_strings[:5])
  print()
  print(example_target_strings[:5])
  break

tf.Tensor(
[b'Yes, we need another idol please.' b'It is education.'
 b'Be sure to review the instructions your computer may be the problem.'
 b'How long ago? A decade?' b"Batman is great. I wouldn't of had guessed."], shape=(5,), dtype=string)

tf.Tensor(
[b'yes, please we need another idol!...'
 b'Oh Crap, Not Another Learning Experience!'
 b'check the instructions... it might be your computer...'
 b'or as in like a decade ago?'
 b'/&#92; Batman is Awesome /&#92; , i would have never guessed.'], shape=(5,), dtype=string)


In [15]:
@tf.keras.utils.register_keras_serializable()
def custom_standardizations(input_data):
  # Split accented characters.
  text = tf_text.normalize_utf8(input_data, 'NFKD')
  text = tf.strings.lower(text)
  # Keep space, a to z, and select punctuation.
  text = tf.strings.regex_replace(text, '[^ a-z.?!,]', '')
  # Add spaces around punctuation.
  text = tf.strings.regex_replace(text, '[.?!,]', r' \0 ')
  # Strip whitespace.
  text = tf.strings.strip(text)

  return tf.strings.join(['[START]', text, '[END]'], separator=' ')

In [16]:
example_text = tf.constant('How old are you!?!!')

print(example_text.numpy())
print(tf_text.normalize_utf8(example_text, 'NFKD').numpy())
print(example_text.numpy().decode())
print(custom_standardizations(example_text).numpy().decode())

b'How old are you!?!!'
b'How old are you!?!!'
How old are you!?!!
[START] how old are you !  ?  !  ! [END]


In [17]:
max_vocab_size = 15000

context_text_processor = tf.keras.layers.TextVectorization(
    standardize=custom_standardizations,
    max_tokens=max_vocab_size,
    ragged=True)
target_text_processor = tf.keras.layers.TextVectorization(
    standardize=custom_standardizations,
    max_tokens=max_vocab_size,
    ragged=True)

In [18]:
context_text_processor.adapt(train_raw.map(lambda context, target: context))

# Here are the first 10 words from the vocabulary:
context_text_processor.get_vocabulary()[:10]

['', '[UNK]', '.', '[START]', '[END]', 'i', ',', 'the', 'is', 'to']

In [19]:
target_text_processor.adapt(train_raw.map(lambda context, target: target))
target_text_processor.get_vocabulary()[:10]

['', '[UNK]', '.', '[START]', '[END]', '!', 'i', ',', 'the', 'a']

In [20]:
example_tokens = context_text_processor(example_context_strings)
example_tokens[:3, :]

<tf.RaggedTensor [[3, 80, 6, 114, 135, 270, 480, 110, 2, 4], [3, 11, 8, 3575, 2, 4],
 [3, 27, 104, 9, 3423, 7, 3852, 38, 506, 133, 27, 7, 558, 2, 4]]>

In [21]:
context_vocab = np.array(context_text_processor.get_vocabulary())
tokens = context_vocab[example_tokens[0].numpy()]
' '.join(tokens)

'[START] yes , we need another idol please . [END]'

In [22]:
def process_text(context, target):
  context = context_text_processor(context).to_tensor()
  target = target_text_processor(target)
  targ_in = target[:,:-1].to_tensor()
  targ_out = target[:,1:].to_tensor()
  return (context, targ_in), targ_out


train_ds = train_raw.map(process_text, tf.data.AUTOTUNE)
val_ds = val_raw.map(process_text, tf.data.AUTOTUNE)

In [23]:
for (ex_context_tok, ex_tar_in), ex_tar_out in train_ds.take(1):
  print(ex_context_tok[0, :10].numpy()) 
  print()
  print(ex_tar_in[0, :10].numpy()) 
  print(ex_tar_out[0, :10].numpy()) 
  

[   3  713   11  209    9    7 1414    2    4    0]

[   3  636   11   45    8  104  169   12    8 1301]
[ 636   11   45    8  104  169   12    8 1301 1156]


In [24]:
"""encoder-decoder"""
UNITS = 256

In [25]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, text_processor, units):
    super(Encoder, self).__init__()
    self.text_processor = text_processor
    self.vocab_size = text_processor.vocabulary_size()
    self.units = units
    
    # The embedding layer converts tokens to vectors
    self.embedding = tf.keras.layers.Embedding(self.vocab_size, units,
                                               mask_zero=True)

    # The RNN layer processes those vectors sequentially.
    self.rnn = tf.keras.layers.Bidirectional(
        merge_mode='sum',
        layer=tf.keras.layers.GRU(units,
                            # Return the sequence and state
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform'))

  def call(self, x):
    shape_checker = ShapeChecker()
    shape_checker(x, 'batch s')

    # 2. The embedding layer looks up the embedding vector for each token.
    x = self.embedding(x)
    shape_checker(x, 'batch s units')

    # 3. The GRU processes the sequence of embeddings.
    x = self.rnn(x)
    shape_checker(x, 'batch s units')

    # 4. Returns the new sequence of embeddings.
    return x

  def convert_input(self, texts):
    texts = tf.convert_to_tensor(texts)
    if len(texts.shape) == 0:
      texts = tf.convert_to_tensor(texts)[tf.newaxis]
    context = self.text_processor(texts).to_tensor()
    context = self(context)
    return context

In [26]:
# Encode the input sequence.
encoder = Encoder(context_text_processor, UNITS)
#ex_context = encoder(ex_context_tok)

#print(f'Context tokens, shape (batch, s): {ex_context_tok.shape}')
#print(f'Encoder output, shape (batch, s, units): {ex_context.shape}')

In [27]:
class CrossAttention(tf.keras.layers.Layer):
  def __init__(self, units, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(key_dim=units, num_heads=2, **kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

  def call(self, x, context):
    shape_checker = ShapeChecker()
 
    shape_checker(x, 'batch t units')
    shape_checker(context, 'batch s units')

    attn_output, attn_scores = self.mha(
        query=x,
        value=context,
        return_attention_scores=True)
    
    shape_checker(x, 'batch t units')
    shape_checker(attn_scores, 'batch heads t s')
    
    # Cache the attention scores for plotting later.
    attn_scores = tf.reduce_mean(attn_scores, axis=1)
    shape_checker(attn_scores, 'batch t s')
    self.last_attention_weights = attn_scores

    x = self.add([x, attn_output])
    x = self.layernorm(x)

    return x

In [28]:
attention_layer = CrossAttention(UNITS)

# Attend to the encoded tokens
embed = tf.keras.layers.Embedding(target_text_processor.vocabulary_size(),
                                  output_dim=UNITS, mask_zero=True)

In [29]:
class Decoder(tf.keras.layers.Layer):
  @classmethod
  def add_method(cls, fun):
    setattr(cls, fun.__name__, fun)
    return fun

  def __init__(self, text_processor, units):
    super(Decoder, self).__init__()
    self.text_processor = text_processor
    self.vocab_size = text_processor.vocabulary_size()
    self.word_to_id = tf.keras.layers.StringLookup(
        vocabulary=text_processor.get_vocabulary(),
        mask_token='', oov_token='[UNK]')
    self.id_to_word = tf.keras.layers.StringLookup(
        vocabulary=text_processor.get_vocabulary(),
        mask_token='', oov_token='[UNK]',
        invert=True)
    self.start_token = self.word_to_id('[START]')
    self.end_token = self.word_to_id('[END]')

    self.units = units


    # 1. The embedding layer converts token IDs to vectors
    self.embedding = tf.keras.layers.Embedding(self.vocab_size,
                                               units, mask_zero=True)

    # 2. The RNN keeps track of what's been generated so far.
    self.rnn = tf.keras.layers.GRU(units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    #self.lstm = tf.keras.layers.LSTM(units,
                                   #return_sequences=True,
                                   #return_state=True,
                                   #recurrent_initializer='glorot_uniform')

    # 3. The RNN output will be the query for the attention layer.
    self.attention = CrossAttention(units)

    # 4. This fully connected layer produces the logits for each
    # output token.
    self.output_layer = tf.keras.layers.Dense(self.vocab_size)

In [30]:
@Decoder.add_method
def call(self,
         context, x,
         state=None,
         return_state=False):  
  shape_checker = ShapeChecker()
  shape_checker(x, 'batch t')
  shape_checker(context, 'batch s units')

  # 1. Lookup the embeddings
  x = self.embedding(x)
  shape_checker(x, 'batch t units')

  # 2. Process the target sequence.
  x, state = self.rnn(x, initial_state=state)
  shape_checker(x, 'batch t units')

  # 3. Use the RNN output as the query for the attention over the context.
  x = self.attention(x, context)
  self.last_attention_weights = self.attention.last_attention_weights
  shape_checker(x, 'batch t units')
  shape_checker(self.last_attention_weights, 'batch t s')

  # Step 4. Generate logit predictions for the next token.
  logits = self.output_layer(x)
  shape_checker(logits, 'batch t target_vocab_size')

  if return_state:
    return logits, state
  else:
    return logits

In [31]:
decoder = Decoder(target_text_processor, UNITS)

In [32]:
"""inference"""
@Decoder.add_method
def get_initial_state(self, context):
  batch_size = tf.shape(context)[0]
  start_tokens = tf.fill([batch_size, 1], self.start_token)
  done = tf.zeros([batch_size, 1], dtype=tf.bool)
  embedded = self.embedding(start_tokens)
  return start_tokens, done, self.rnn.get_initial_state(embedded)[0]

In [33]:
@Decoder.add_method
def tokens_to_text(self, tokens):
  words = self.id_to_word(tokens)
  result = tf.strings.reduce_join(words, axis=-1, separator=' ')
  result = tf.strings.regex_replace(result, '^ *\[START\] *', '')
  result = tf.strings.regex_replace(result, ' *\[END\] *$', '')
  return result

In [34]:
@Decoder.add_method
def get_next_token(self, context, next_token, done, state, temperature = 0.0):
  logits, state = self(
    context, next_token,
    state = state,
    return_state=True) 
  
  if temperature == 0.0:
    next_token = tf.argmax(logits, axis=-1)
  else:
    logits = logits[:, -1, :]/temperature
    next_token = tf.random.categorical(logits, num_samples=1)

  # If a sequence produces an `end_token`, set it `done`
  done = done | (next_token == self.end_token)
  # Once a sequence is done it only produces 0-padding.
  next_token = tf.where(done, tf.constant(0, dtype=tf.int64), next_token)
  
  return next_token, done, state

In [35]:
"""The model"""
class Translator(tf.keras.Model):
  @classmethod
  def add_method(cls, fun):
    setattr(cls, fun.__name__, fun)
    return fun

  def __init__(self, units,
               context_text_processor,
               target_text_processor):
    super().__init__()
    # Build the encoder and decoder
    encoder = Encoder(context_text_processor, units)
    decoder = Decoder(target_text_processor, units)

    self.encoder = encoder
    self.decoder = decoder

  def call(self, inputs):
    context, x = inputs
    context = self.encoder(context)
    logits = self.decoder(context, x)

    #TODO(b/250038731): remove this
    try:
      # Delete the keras mask, so keras doesn't scale the loss+accuracy. 
      del logits._keras_mask
    except AttributeError:
      pass

    return logits

In [36]:
model = Translator(UNITS, context_text_processor, target_text_processor)
# model = tf.keras.models.load_model("/content/drive/MyDrive/model_rnn_attention")

In [37]:
logits = model((ex_context_tok, ex_tar_in))

print(f'Context tokens, shape: (batch, s, units) {ex_context_tok.shape}')
print(f'Target tokens, shape: (batch, t) {ex_tar_in.shape}')
print(f'logits, shape: (batch, t, target_vocabulary_size) {logits.shape}')

Context tokens, shape: (batch, s, units) (16, 24)
Target tokens, shape: (batch, t) (16, 24)
logits, shape: (batch, t, target_vocabulary_size) (16, 24, 15000)


In [38]:
def masked_loss(y_true, y_pred):
    # Calculate the loss for each item in the batch.
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')
    loss = loss_fn(y_true, y_pred)

    # Mask off the losses on padding.
    mask = tf.cast(y_true != 0, loss.dtype)
    loss *= mask

    # Return the total.
    return tf.reduce_sum(loss)/tf.reduce_sum(mask)

In [39]:
def masked_acc(y_true, y_pred):
    # Calculate the loss for each item in the batch.
    y_pred = tf.argmax(y_pred, axis=-1)
    y_pred = tf.cast(y_pred, y_true.dtype)
    
    match = tf.cast(y_true == y_pred, tf.float32)
    mask = tf.cast(y_true != 0, tf.float32)
    
    return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [40]:
model.compile(optimizer='adam',
              loss=masked_loss, 
              metrics=[masked_acc, masked_loss])

In [41]:
vocab_size = 1.0 * target_text_processor.vocabulary_size()

{"expected_loss": tf.math.log(vocab_size).numpy(),
 "expected_acc": 1/vocab_size}

{'expected_loss': 9.615806, 'expected_acc': 6.666666666666667e-05}

In [42]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

In [48]:
history = model.fit(
    train_ds.repeat(), 
    epochs=20,
    steps_per_epoch = 200,
    validation_data=val_ds,
    validation_steps = 60
)

Epoch 1/20
200/200 [==============================] - 83s 414ms/step - loss: 2.6641 - masked_acc: 0.5125 - masked_loss: 2.6641 - val_loss: 3.0184 - val_masked_acc: 0.4773 - val_masked_loss: 3.0184
Epoch 2/20
200/200 [==============================] - 83s 417ms/step - loss: 2.6891 - masked_acc: 0.5090 - masked_loss: 2.6891 - val_loss: 3.0749 - val_masked_acc: 0.4664 - val_masked_loss: 3.0749
Epoch 3/20
200/200 [==============================] - 79s 397ms/step - loss: 2.6681 - masked_acc: 0.5101 - masked_loss: 2.6681 - val_loss: 3.1859 - val_masked_acc: 0.4548 - val_masked_loss: 3.1859
Epoch 4/20
200/200 [==============================] - 80s 398ms/step - loss: 2.6408 - masked_acc: 0.5149 - masked_loss: 2.6408 - val_loss: 2.9690 - val_masked_acc: 0.4877 - val_masked_loss: 2.9690
Epoch 5/20
200/200 [==============================] - 79s 396ms/step - loss: 2.7199 - masked_acc: 0.5046 - masked_loss: 2.7199 - val_loss: 3.1228 - val_masked_acc: 0.4568 - val_masked_loss: 3.1228
Epoch 6/20
200/

In [69]:
import os
model.save("/notebooks/GYAFC_Corpus/saved_models/model",save_format='tf')
#model.save("/content/drive/MyDrive/model_rnn_attention",save_format='tf')
model.save_weights('/notebooks/GYAFC_Corpus/saved_models/my_weights')

In [50]:
#customizsed function loaded again when the model is reloaded.
@tf.keras.utils.register_keras_serializable()
def custom_standardizations(input_data):
  # Split accented characters.
  text = tf_text.normalize_utf8(input_data, 'NFKD')
  text = tf.strings.lower(text)
  # Keep space, a to z, and select punctuation.
  text = tf.strings.regex_replace(text, '[^ a-z.?!,]', '')
  # Add spaces around punctuation.
  text = tf.strings.regex_replace(text, '[.?!,]', r' \0 ')
  # Strip whitespace.
  text = tf.strings.strip(text)

  return tf.strings.join(['[START]', text, '[END]'], separator=' ')
#model.load_weights('/notebooks/GYAFC_Corpus/saved_models/my_weights')
#model = tf.keras.models.load_model('path/to/model')

ValueError: Custom>custom_standardizations has already been registered to <function custom_standardizations at 0x00000274E8D7E320>

In [51]:
#@title
@Translator.add_method
def translate(self,
              texts, *,
              max_length=50,
              temperature=0.0):
  # Process the input texts
  context = self.encoder.convert_input(texts)
  batch_size = tf.shape(texts)[0]

  # Setup the loop inputs
  tokens = []
  attention_weights = []
  next_token, done, state = self.decoder.get_initial_state(context)

  for _ in range(max_length):
    # Generate the next token
    next_token, done, state = self.decoder.get_next_token(
        context, next_token, done,  state, temperature)
        
    # Collect the generated tokens
    tokens.append(next_token)
    attention_weights.append(self.decoder.last_attention_weights)
    
    if tf.executing_eagerly() and tf.reduce_all(done):
      break

  # Stack the lists of tokens and attention weights.
  tokens = tf.concat(tokens, axis=-1)   # t*[(batch 1)] -> (batch, t)
  self.last_attention_weights = tf.concat(attention_weights, axis=1)  # t*[(batch 1 s)] -> (batch, t s)

  result = self.decoder.tokens_to_text(tokens)
  return result

In [56]:
result = model.translate(["Your mother is unintelligent. I am not stupid. Never gonna give you up"]) # Are you still home
result[0].numpy().decode()

'yo so stupid , im not stupid . . . never gonna give you up . '

In [58]:
result = model.translate(["No, I apologize; all I know is that it was a good movie"]) # Are you still home
result[0].numpy().decode()

'no sorry all i know is that it was a good movie '

In [67]:
result = model.translate(["It depends on how much money and drugs you have"]) # Are you still home
result[0].numpy().decode()

'depends on how much money and psp you have '